# Real Estate Project

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

Load the data into a dataframe

In [2]:
data = pd.read_csv('data_folder/kc_house_data.csv', index_col = 0)
data

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,3,8,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,3,8,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,3,7,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007


In [3]:
data.shape

(21597, 20)

Count the null values

In [4]:
data.isna().sum()

date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

Fill the null values with yr_renovated with 0 meaning it was not renovated.

In [5]:
data['yr_renovated'] = data["yr_renovated"].fillna(0)

Drop what are essentially null values from sqft_basement losing less than 2% of the data.

In [6]:
data = data[~data['sqft_basement'].isin(['?'])]
data.shape

(21143, 20)

Round bathrooms to the nearest half-bath

In [7]:
data['bathrooms']=round(data['bathrooms']*2)/2

<ipython-input-7-67c3fce2bafa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bathrooms']=round(data['bathrooms']*2)/2


In [8]:
data['bathrooms'].value_counts()

2.0    6891
2.5    5255
1.0    3837
3.0    2470
1.5    1422
3.5     719
4.0     366
4.5      96
5.0      55
6.0      12
5.5       9
8.0       3
0.5       3
6.5       2
7.0       2
7.5       1
Name: bathrooms, dtype: int64

Convert 'sqft_basement' into a float type in order to create a new variable 'has_basement'

In [9]:
data['sqft_basement']=data['sqft_basement'].astype(float, copy=False)

<ipython-input-9-165e36304c78>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sqft_basement']=data['sqft_basement'].astype(float, copy=False)


In [10]:
data['has_basement']= np.where(data['sqft_basement']!= 0, True, False)

<ipython-input-10-1df61e2bf4a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['has_basement']= np.where(data['sqft_basement']!= 0, True, False)


In [11]:
data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,has_basement
id,,,,,,,,,,,,,,,,,,,,,
7129300520,10/13/2014,221900.0,3,1.0,1180,5650,1.0,NaN,0.0,3,...,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,False
6414100192,12/9/2014,538000.0,3,2.0,2570,7242,2.0,0.0,0.0,3,...,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,True
5631500400,2/25/2015,180000.0,2,1.0,770,10000,1.0,0.0,0.0,3,...,770,0.0,1933,0.0,98028,47.7379,-122.233,2720,8062,False
2487200875,12/9/2014,604000.0,4,3.0,1960,5000,1.0,0.0,0.0,5,...,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,True
1954400510,2/18/2015,510000.0,3,2.0,1680,8080,1.0,0.0,0.0,3,...,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503,False


Create a new variable called 'is_renovated' 

In [12]:
data['is_renovated']= np.where(data['yr_renovated']!= 0, True, False)

<ipython-input-12-534eb58d893c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['is_renovated']= np.where(data['yr_renovated']!= 0, True, False)


We are interested to see if seasonality will play a role in the housing price. Change the date to a datetime column, isolate the month, and code for each season.

In [13]:
data['date'] = pd.to_datetime(data['date'])
data['month']= data['date'].dt.month
data = data.assign(season=lambda x: x["date"].apply(lambda s: ["winter","spring","summer","fall"][(s.month-1)//3]))

<ipython-input-13-b9c90b28d6e8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'])
<ipython-input-13-b9c90b28d6e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['month']= data['date'].dt.month


In [ ]:
data.info()

Drop variables that will not be apart of the model.

In [ ]:
data.drop(['waterfront','view', 'sqft_above', 'zipcode','month', 'yr_renovated','sqft_basement'], axis=1, inplace=True)

In [ ]:
data.columns

Create a scatterplot for each variable in relationship to price to investigate whether variable are categorical.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,3))

for xcol, ax in zip(['date', 'bedrooms', 'bathrooms', 'sqft_living'], axes):
    data.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.4, color='b')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,3))

for xcol, ax in zip(['sqft_lot', 'floors', 'condition', 'grade'], axes):
    data.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.4, color='b')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,3))

for xcol, ax in zip(['yr_built', 'lat', 'long', 'sqft_living15'], axes):
    data.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.4, color='b')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,3))

for xcol, ax in zip(['sqft_lot15','season'], axes):
    data.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.4, color='b')

In [ ]:
#pd.plotting.scatter_matrix(data,figsize  = [20, 20]);
#plt.show()

Create list of discrete variables

In [ ]:
discrete_features = ['bedrooms', 'bathrooms', 'floors', 'condition', 'grade', 'is_renovated','has_basement', 'season']

Plot a bar graph for each discrete variable.

In [ ]:
for feature in discrete_features:
    plt.figure(figsize=(10,8))
    sns.barplot(x=feature, y='price', data=data)

In [ ]:
data.drop('season', axis=1, inplace=True)

Create a list of the continuous variables.

In [ ]:
continuous_features = [feature for feature in data.columns if feature not in discrete_features+['price']+['date']]
continuous_features

Plot the distribution for each of the continuous variables

In [ ]:
for feature in continuous_features:
    data[feature].plot.hist(density=True, label = feature+' histogram')
    data[feature].plot.kde(label =feature+' kde')
    plt.legend()
    plt.show()

Dealing with Outliers

In [ ]:
def lower_limit(col):
    lower_limit = np.mean(col)-3*np.std(col)
    return lower_limit

In [ ]:
def upper_limit(col):
    upper_limit = np.mean(col)+3*np.std(col)
    return upper_limit

Removing outliers from continuous variables

In [ ]:
data = data[(data['sqft_living'] >= lower_limit(data['sqft_living'])) & (data['sqft_living'] <= upper_limit(data['sqft_living']))]

In [ ]:
data = data[(data['sqft_lot'] >= lower_limit(data['sqft_lot'])) & (data['sqft_lot'] <= upper_limit(data['sqft_lot']))]

In [ ]:
data = data[(data['sqft_living15'] >= lower_limit(data['sqft_living15'])) & (data['sqft_living15'] <= upper_limit(data['sqft_living15']))]

In [ ]:
data = data[(data['sqft_lot15'] >= lower_limit(data['sqft_lot15'])) & (data['sqft_lot15'] <= upper_limit(data['sqft_lot15']))]

In [ ]:
data.shape

Remove outlier from bedroom data (33 bedrooms with 1620 sqft of living space)

In [ ]:
data['bedrooms'].max()

In [ ]:
data[(data['bedrooms']==33)]

In [ ]:
data = data[(data['bedrooms']!=33)]

Look at the distribution for the continuous variables after removing outliers.

In [ ]:
for feature in continuous_features:
    data[feature].plot.hist(density=True, label = feature+' histogram')
    data[feature].plot.kde(label =feature+' kde')
    plt.legend()
    plt.show()

Look at the correlations of each columns with price.

In [ ]:
price_corrs = data.corr()["price"].map(abs).sort_values(ascending=False)
price_corrs

Create a heat map to identify correlation between variables to look for issues with multicollinearity.

In [ ]:
cmap = sns.diverging_palette(10, 240, as_cmap=True)
plt.figure(figsize=(15, 12))
heatmap = sns.heatmap(data.corr(), vmin=-1, vmax=1, annot=True, cmap=cmap)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12);

Drop the sqft_living15 and sqft_lot15 as they are highly correlated with sqft_living and sqft_lot. 

In [ ]:
data.drop(labels = ['sqft_living15','sqft_lot15'], axis=1, inplace=True)

__Create dummy variables__

In [ ]:
dummy = pd.get_dummies(data, columns=['is_renovated', 'has_basement'], drop_first=True)
dummy.drop(labels=['date'], axis=1, inplace=True)
dummy

__Train-Test Split__

In [ ]:
y = dummy['price']
X = dummy.drop(['price'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

__Regression #1 : Using Standardized Variables__

Standard scale all variables

In [ ]:
X_train_scaled = (X_train - np.mean(X_train))/ np.std(X_train)
y_train_scaled = (y_train - np.mean(y_train))/ np.std(y_train)

In [ ]:
X_test_scaled = (X_test - np.mean(X_test))/ np.std(X_test)
y_test_scaled = (y_test - np.mean(y_test))/ np.std(y_test)

In [ ]:
X_train_scaled.hist(figsize=[10,10]);

In [ ]:
y_train_scaled.hist(figsize=[6,4]);

In [ ]:
pred = sm.add_constant(X_train_scaled)
model = sm.OLS(endog=y_train_scaled, exog=pred).fit()
model.summary()

Evaluate the fit and prediction with the training dataset

In [ ]:
X_train1 = X_train_scaled
y_train1 = y_train_scaled
X_test1 = X_test_scaled
y_test1 = y_test_scaled

lr = LinearRegression()
lr.fit(X_train1, y_train1)

y_train_hat1 = lr.predict(X_train1)
mse_train1 = mean_squared_error(y_train1, y_train_hat1)
rmse_train1 = mean_squared_error(y_train1, y_train_hat1, squared=False)
r2_train1 = lr.score(X_train1, y_train1)

print(f'Evaluate Model Prediction with Training Dataset:')
print('------------------------------------------------')
print(f'Mean Squared Error: {mse_train1}')
print(f'Root Mean Squared Error: {rmse_train1}')
print(f'R^2: {r2_train1}')

Validate the model with the testing dataset

In [ ]:
import scipy.stats as stats
residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
fig.show()

In [ ]:
vif = [variance_inflation_factor(df_model_1.values, i) for i in range(df_model_1.shape[1])]
X1_cols = list(df_model_1.columns)

list(zip(X1_cols, vif))

__Regression #2 : Using Log-Scaled & Standardized Variables__

Drop lat and long since ...

In [ ]:
X_train_log = pd.DataFrame.copy(X_train)

In [ ]:
X_train_log.drop(['lat','long'], axis=1, inplace=True)

In [ ]:
X_train_log

In [ ]:
log_col = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'condition', 'grade', 'yr_built']
for col in X_train.columns:
    if col in log_col:
        new_log = np.log(X_train_log[col])
        X_train_log[col] = new_log


In [ ]:
y_train_log = np.log(y_train)

In [ ]:
X_train_log_scaled = (X_train_log - np.mean(X_train_log))/ np.std(X_train_log)
y_train_log_scaled = (y_train_log - np.mean(y_train_log))/ np.std(y_train_log)

In [ ]:
X_train_log_scaled.hist(figsize=[10,10]);

In [ ]:
y_train_log_scaled.hist(figsize=[6,4]);

In [ ]:
pred2 = sm.add_constant(X_train_log_scaled)
model2 = sm.OLS(endog=y_train_log_scaled, exog=pred2).fit()
model2.summary()

In [ ]:
X_test_log = pd.DataFrame.copy(X_test)

In [ ]:
X_test_log.drop(['lat','long'], axis=1, inplace=True)

In [ ]:
log_col = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'condition', 'grade', 'yr_built']
for col in X_test.columns:
    if col in log_col:
        new_log = np.log(X_test_log[col])
        X_test_log[col] = new_log


In [ ]:
y_test_log = np.log(y_test)

In [ ]:
X_test_log_scaled = (X_test_log - np.mean(X_test_log))/ np.std(X_test_log)
y_test_log_scaled = (y_test_log - np.mean(y_test_log))/ np.std(y_test_log)

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train_log_scaled, y_train_log_scaled)

y_hat_log_scaled_train = linreg.predict(X_train_log_scaled)
y_hat_log_scaled_test = linreg.predict(X_test_log_scaled)

train_log_scaled_mse = mean_squared_error(y_train_log_scaled, y_hat_log_scaled_train)
test_log_scaled_mse = mean_squared_error(y_test_log_scaled, y_hat_log_scaled_test)
print('Train Mean Squarred Error:', train_log_scaled_mse)
print('Test Mean Squarred Error:', test_log_scaled_mse)

In [ ]:
import scipy.stats as stats
residuals = model2.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
fig.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model2, "sqft_lot", fig=fig)
plt.show()

__Regression #3 : Recursive Feature Elimination__

In [ ]:
from sklearn.feature_selection import RFE

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select=5)
selector = selector.fit(pred2, y_train_log_scaled)

In [ ]:
selector.support_ 

In [ ]:
pred2.columns

In [ ]:
estimators = selector.estimator_
print(estimators.coef_)
print(estimators.intercept_)

In [ ]:
pred2_columns = list(pred2.columns)
tf = list(selector.support_)
filtered = np.array(pred2_columns)[np.array(tf)]
filtered
pred3 = pred2[filtered]
pred3

In [ ]:
pred3 = sm.add_constant(pred3)
model3 = sm.OLS(endog=y_train_log_scaled, exog=pred3).fit()
model3.summary()

Visualizations of results

In [ ]:
df_graph = dummy.groupby('bathrooms')['price'].mean()
df_graph=pd.DataFrame(df_graph)
df_graph.reset_index(inplace=True)
df_graph.loc[len(df_graph.index)] = ['6.5', 0] 
df_graph.loc[len(df_graph.index)] = ['7.0', 0] 
df_graph['bathrooms']=df_graph['bathrooms'].astype(float, copy=False)

In [ ]:
plt.figure(figsize=(8,6))
ax = sns.barplot(x='bathrooms', y='price', data=df_graph, palette="bright")
ax.set_title('Mean Price by Bathrooms')
ax.set_xlabel('Number of Bathrooms')
ax.set_ylabel('Price (in Millions)')
plt.savefig('Bathrooms.png');
#ticks = np.arange(0,15,0.5)
#ax.set_xticks(ticks);

In [ ]:
plt.figure(figsize=(8,6))
ax = sns.barplot(x='grade', y='price', data=dummy,palette='bright')
ax.set_title('Mean Price by Grade')
ax.set_xlabel('Grade')
ax.set_ylabel('Price (in Millions)')
plt.savefig('Grade.png');

Playing with latitude and longitude graphing

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = sns.scatterplot(x=data["long"], y=data["lat"], hue=data['price'], palette="inferno",
                     marker=".")
ax.set( xlabel="Longitude",
        ylabel="Latitude", 
        title="Price by Location");